# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS207 Systems Development for Computational Science: 
## Milestone 2 - Implementation Demo



**Harvard University**<br/>
**Fall 2018**<br/>
**Team Members**: Will Claybaugh, Erin Williams, and Bruce Xiong

<hr style="height:2pt">

For powers/roots/exponential functions, the generic form is as the following:<br/>
$$f(x)=\left[u(x)\right]^{v(x)}$$

Therefore when we implement the derivatives, including the very special case such as $y=x^x$, the following chain rule applies:<br/>
$$\frac{df(x)}{dx}=\frac{d\left[u(x)\right]^{v(x)}}{dx}=v(x)\left[u(x)\right]^{v(x)-1}\cdot u'(x)+\left[u(x)\right]^{v(x)}\cdot ln(u(x))\cdot v'(x)$$

In [2]:
# powers/roots/exponential
import numpy as np
class autoDiff():
    def __init__(self, a):
        # Constructor to set up dual number
        self.val = a
        self.deriv = 1
        
    def __add__(self, other):
        try:
            y = autoDiff(self.val + other.val)
            y.deriv = self.deriv + other.deriv
        except AttributeError:
            y = autoDiff(self.val + other)
            y.deriv = self.deriv
        return y
    
    __radd__ = __add__
    
    def __sub__(self, other):
        try:
            y = autoDiff(self.val - other.val)
            y.deriv = self.deriv - other.deriv
        except AttributeError:
            y = autoDiff(self.val - other)
            y.deriv = self.deriv
        return y
            
    def __rsub__(self, other):
        try:
            y = autoDiff(self.val - other.val)
            y.deriv = self.deriv - other.deriv
        except AttributeError:
            y = autoDiff(other - self.val)
            y.deriv = -self.deriv
        return y
    
    def __mul__(self, other):
        try:
            y = autoDiff(self.val * other.val)
            y.deriv = self.val * other.deriv + self.deriv * other.val
        except AttributeError:
            y = autoDiff(self.val * other)
            y.deriv = self.deriv * other
        return y
    
    __rmul__ = __mul__
    
    def __truediv__(self, other):
        try:
            y = autoDiff(self.val/other.val)
            y.deriv = (self.deriv*other.val - self.val*other.deriv)/((other.val)**2)
        except AttributeError:
            y = autoDiff(self.val/other)
            y.deriv = self.deriv/other
        return y
    
    def __rtruediv__(self, other):
        try:
            y = autoDiff(self.val/other.val)
            y.deriv = (self.deriv*other.val - self.val*other.deriv)/((other.val)**2)
        except AttributeError:
            y = autoDiff(other/self.val)
            y.deriv = -other/((self.val)**2)*self.deriv
        return y            
    
    def __neg__(self):
        try:
            y = autoDiff(-self.val)
            y.deriv = -self.deriv
        except AttributeError:
            y = autoDiff(-self)
            y.deriv = -self
        return y
    
    def __pow__(self, other): #self^other => self = u(x) and other = v(x)
        try:
            y = autoDiff(self.val**(other.val))
            y.deriv = other.val*((self.val)**(other.val-1))*self.deriv + ((self.val)**(other.val))*(np.log(self.val))*other.deriv
        except AttributeError: #x^a:
            y = autoDiff(self.val**other)
            y.deriv = other*((self.val)**(other-1))*self.deriv
        return y
    
    def exp(other, self):
        try: # we may not necesarily implement this try part
            y = autoDiff(other.val**(self.val))
            y.deriv = other.val*((other.val)**(self.val-1))*other.deriv + ((other.val)**(self.val))*(np.log(other.val))*self.deriv
        except AttributeError:
            y = autoDiff(other**(self.val))
            y.deriv = other**(self.val)*np.log(other)*self.deriv
        return y
    
    def log(other, self):
        try:
            y = autoDiff(np.log(self.val)/np.log(other.val))
            y.deriv = (self.deriv*np.log(other.val)/self.val - other.deriv*np.log(self.val)/other.val)/(np.log(other.val)**2)
        except AttributeError:
            y = autoDiff(np.log(self.val)/np.log(other))
            y.deriv = 1/self.val/np.log(other)*self.deriv                
        return y
        
    def logx(self, other):#when base is a function of x
        try:
            y = autoDiff(np.log(other.val)/np.log(self.val))
            y.deriv = (other.deriv*np.log(self.val)/other.val - self.deriv*np.log(other.val)/self.val)/(np.log(self.val)**2)
        except AttributeError:
            y = autoDiff(np.log(other)/np.log(self.val))
            y.deriv = -np.log(other)*self.deriv/self.val/((np.log(self.val))**2)
        return y
        
    def sin(self):
        try:
            y = autoDiff(np.sin(self.val))
            y.deriv = np.cos(self.val)*self.deriv
        except AttributeError:
            y = autoDiff(np.sin(self))
            y.deriv = 0
        return y
        
    def cos(self):
        try:
            y = autoDiff(np.cos(self.val))
            y.deriv = -np.sin(self.val)*self.deriv
        except AttributeError:
            y = autoDiff(np.cos(self))
            y.deriv = 0
        return y
        
    def tan(self): #need to check 0 in denominator
        a = autoDiff.sin(self)
        b = autoDiff.cos(self)
        y = a/b
        return y
    
    def cot(self):
        a = autoDiff.tan(self)
        y = 1/a
        return y
    
    def sec(self):
        a = autoDiff.sin(self)
        y = 1/a
        return y
    
    def csc(self):
        a = autoDiff.cos(self)
        y = 1/a
        return y
    
    def arcsin(self): # must make sure self is strictly between -1 and 1, exclusive
        if type(self)==autoDiff and (self.val<-1 or self.val>1): # out of domain
            raise Exception('The value entering into arcsin function must be strictly within -1 and 1.')
            
        if type(self)!=autoDiff and (self<-1 or self>1):
            raise Exception('The value entering into arcsin function must be strictly within -1 and 1.')
            
        try:
            y = autoDiff(np.arcsin(self.val))
            y.deriv = 1/((1-self.val**2)**(0.5))*self.deriv
        except AttributeError:
            y = autoDiff(np.arcsin(self))
            y.deriv = 0
        return y
            
    def arccos(self):
        a = autoDiff.arcsin(self)
        y = np.pi/2 - a
        return y
    
    def arctan(self):
        a = self/((1+self**2)**0.5)
        y = autoDiff.arcsin(a)
        return y
    
    def arccot(self):
        a = autoDiff.arctan(self)
        y = np.pi/2 - a
        return y
    
    def arcsec(self):
        a = 1/self
        y = autoDiff.arccos(a)
        return y
    
    def arccsc(self):
        a = autoDiff.arcsec(self)
        y = np.pi/2 - a
        return y
    
    #### Do we need to implement hyperbolic function e.g. sinh, cosh, tanh, coth, etc.? ####
        

logarithm can be treated as inverse function of exponential function with the same base:<br/>
$$f(x)=a^x\rightarrow f^{-1}(x)=log_a(x)$$
So we have:<br/>
$$\frac{df^{-1}(x)}{dx}=\frac{dlog_a(x)}{dx}=\frac{1}{a^{f^{-1}(x)}ln(a)}=\frac{1}{xln(a)}$$

Note that an extreme case in logarithm function is in the form of: $f(x)=\log_{U(x)}V(x)$, where the derivative is:
$$f'(x)=\left[\frac{\ln(V(x))}{\ln(U(x))}\right]'=\frac{\frac{V'(x)\ln(U(x))}{V(x)}-\frac{U'(x)\ln(V(x))}{U(x)}}{ln^2(U(x))}$$

Some rules (there are more!) regarding trigonometric and inverse trigonometric functions:
- $\tan(x)=\frac{sin(x)}{cos(x)}$
- $\sec(x)=\frac{1}{cos(x)}$
- $\arccos(x)=\frac{\pi}{2}-\arcsin(x)$, where x$\in$(-1,1)
- ...

In [3]:
a = 2.0; b = 5.8; c = 3.0; d = -2.75; e = 4.0; inc = 0.0000001

Test Case #1:<br/>
$$f(x)=-4x^{5.8} + 3$$

In [7]:
x = autoDiff(a)
f = -e*x**b + c
print(f.val, f.deriv)
g0 = -e*a**b + c
g1 = -e*(a+inc)**b + c
print(g0, (g1-g0)/inc)

-219.86094420380775 -646.2967381910424
-219.86094420380775 -646.2968147502579


<hr style="height:2pt">

Test Case #2:<br/>
$$f(x)=-2.75\cdot 5.8^x\cdot x^3-\frac{\cos(x)}{x}$$

In [12]:
x = autoDiff(a)
f = d*autoDiff.exp(b,x) * x**c-autoDiff.cos(x)/x
print(f.val, f.deriv)
g0 = d*(b**a) * a**c-np.cos(a)/a
g1 = d*(b**(a+inc)) * (a+inc)**c-np.cos(a1)/a1
print(g0, (g1-g0)/inc)

-739.8719265817265 -2410.7248756178847
-739.8719265817265 -2410.7252363592124


<hr style="height:2pt">

Test Case #3:<br/>
$$f(x)=(3x+\log_x(5.8))^{\sqrt{x}+\frac{1}{x}}$$

In [13]:
x = autoDiff(a)
f = (c*x + autoDiff.logx(x,b))**(x**(1/2)+1/x)
print(f.val, f.deriv)
g0 = (c*a + np.log(b)/np.log(a))**(a**(1/2)+1/a)
a1 = a + inc
g1 = (c*a1 + np.log(b)/np.log(a1))**(a1**(1/2)+1/a1)
print(g0, (g1-g0)/inc)

60.621261109269476 29.37478740554978
60.621261109269476 29.374791523650856


<hr style="height:2pt">

Test Case #4:<br/>
$$f(x)=x-\exp\left(-2\sin^2(4x)\right)$$

In [30]:
x = autoDiff(a)
bs = np.exp(1)
f = x-autoDiff.exp(bs,-2*autoDiff.sin(4*x)**2)
print(f.val, f.deriv)
g0 = a-np.exp(-2*np.sin(4*a)**2)
a1 = a + inc
g1 = a1-np.exp(-2*np.sin(4*a1)**2)
print(g0, (g1-g0)/inc)

1.858811511058501 0.6748109260705084
1.858811511058501 0.674810454182051


<hr style="height:2pt">

Test Case #5:<br/>
$$f(x)=\exp\left(-\sqrt{x+\cos^2(3x)}\right)\sin\left(x\ln(1+x^2)\right)$$

In [4]:
x = autoDiff(a)
bs = np.exp(1)
f = autoDiff.exp(bs,-(x+autoDiff.cos(3*x)**2)**0.5)*autoDiff.sin(x*autoDiff.log(bs,1+x**2))
print(f.val, f.deriv)
g0 = np.exp(-(a+np.cos(3*a)**2)**0.5)*np.sin(a*np.log(1+a**2))
a1 = a + inc
g1 = np.exp(-(a1+np.cos(3*a1)**2)**0.5)*np.sin(a1*np.log(1+a1**2))
print(g0, (g1-g0)/inc)

-0.013972848911640818 -0.5684464955717082
-0.013972848911640818 -0.5684464572837389
